## Setup

In [2]:
from os import getcwd
from os.path import dirname
from IPython.display import clear_output
from ultralytics import checks

ROOT = dirname(getcwd())
clear_output()
checks()

Ultralytics YOLOv8.1.47 🚀 Python-3.11.5 torch-2.2.2 CPU (Apple M3 Max)
Setup complete ✅ (16 CPUs, 64.0 GB RAM, 669.1/1858.2 GB disk)


## Prepare Custom Dataset

1. Create or login to a Roboflow [account](https://app.roboflow.com/login)
2. Create a new project in the Roboflow [dashboard](https://app.roboflow.com)
3. Choose "Object Detection" for Project Type
4. [Add dataset images to project](https://docs.roboflow.com/adding-data/object-detection); upload each class in separate batches, and tag batch with class name
5. Label images with [Roboflow Annotate](https://docs.roboflow.com/annotate) and divide the work evenly amongst teammates
6. Go to Versions, include preprocessing and augmentations, and generate a new dataset

## Import Existing Dataset

Create new directory to store dataset.

In [ ]:
from os import mkdir
from os.path import join, exists

DATASET_ROOT = join(ROOT, "datasets")
if not exists(DATASET_ROOT): mkdir(DATASET_ROOT)

Replace constants with your Roboflow API Key, workspace name, project name, and dataset version respectively.

In [1]:
API_KEY = "OZYVRHRpfy8DaxBPca6s"
WORKSPACE = "capstone2algae"
PROJECT_NAME = "algae-detection-1opyx"
VERSION = 10

Download dataset into datasets directory.

In [2]:
%cd {DATASET_ROOT}

from roboflow import Roboflow

PROJECT = Roboflow(api_key = API_KEY).workspace(WORKSPACE).project(PROJECT_NAME)
PROJECT_VERSION = PROJECT.version(VERSION)
DATASET = PROJECT_VERSION.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.47, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to algae-detection-10 in yolov8:: 100%|██████████| 2024/2024 [00:00<00:00, 12630.33it/s]

/Users/kiran/Documents/workspace/Projects/algae-detection/src/algae-detection-10


## Custom Training

In [ ]:
from torch import device
from torch.cuda import is_available as is_cuda_available
from torch.backends.mps import is_available as is_mps_available
from ultralytics import YOLO

DATASET_PATH = DATASET.location
DATASET_CFG = join(DATASET_PATH, "data.yaml")
CUSTOM_MODEL = YOLO(model = "yolov8n", task = "detect")
DEVICE = device("mps" if is_mps_available() else "cuda" if is_cuda_available() else "cpu")
CONFIDENCE = 0.25
EPOCHS = 25
IMG_SIZE = 256

[Train YOLOv8](https://docs.ultralytics.com/modes/train) on [Detect](https://docs.ultralytics.com/tasks/detect) datasets. Model is automatically validated after training is finished, unless toggled off.

In [ ]:
%cd {ROOT}

CUSTOM_MODEL.train(data = DATASET_CFG, plots = True, epochs = EPOCHS, imgsz = IMG_SIZE, device = DEVICE)

After training, view contents of directory containing saved training results. Newly trained weights are located at `{ROOT}/runs/detect/train/weights/best.pt`.

In [ ]:
TRAIN_DIR = join(ROOT, "runs", "detect", "train")

%ls {TRAIN_DIR}

Helper function to display images.

In [3]:
from IPython.display import Image, display

def output_img(path: str, width: int = 600, height: int = 600) -> None:
    display(Image(filename = path, width = width, height = height))

Output some charts within the aforementioned directory.

In [ ]:
output_img(join(TRAIN_DIR, "confusion_matrix.png"))

In [ ]:
output_img(join(TRAIN_DIR, "results.png"))

In [ ]:
output_img(join(TRAIN_DIR, "val_batch0_pred.jpg"))

## Validate Model

Note that the model is automatically validated after training is finished (unless toggled off), so this step is optional.

In [ ]:
CUSTOM_MODEL.val(data = DATASET_CFG, imgsz = IMG_SIZE, conf = CONFIDENCE, device = DEVICE)

## Inference with Custom Model

Helper function to choose random test image.

In [ ]:
from os import listdir
from random import choice

def rand_test_img(ds_path: str = DATASET_PATH) -> str:
  test_dir = join(ds_path, "test", "images")
  rand_img = choice(listdir(test_dir))
  print("Running inference on", rand_img)

  return join(test_dir, rand_img)

### Standard Model

In [ ]:
CUSTOM_MODEL.predict(rand_test_img(), save = True, imgsz = IMG_SIZE, conf = CONFIDENCE, device = DEVICE)

### SAHI Model

In [ ]:
from cv2 import imread
from numpy import ndarray
from supervision import Detections, InferenceSlicer, LabelAnnotator, BoundingBoxAnnotator, plot_image

def callback(image: ndarray) -> Detections:
    return Detections.from_ultralytics(CUSTOM_MODEL(image, device = DEVICE, conf = CONFIDENCE)[0])

test_img = imread(rand_test_img())
slicer = InferenceSlicer(callback = callback)
detections = slicer(image = test_img)
annotated_img = BoundingBoxAnnotator().annotate(scene = test_img.copy(), detections = detections)

plot_image(image = LabelAnnotator().annotate(scene = annotated_img, detections = detections))

## Export Model

Export a YOLOv8 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://docs.ultralytics.com/integrations/onnx/) or [OpenVINO](https://docs.ultralytics.com/integrations/openvino/) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://docs.ultralytics.com/integrations/tensorrt/) for up to 5x GPU speedup.

| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/)                              | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`, `int8`                             |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`, `int8`                            |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`, `half`, `int8`                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [NCNN](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |

In [ ]:
CUSTOM_MODEL.export()

## Deploy model on Roboflow

Newly trained weights can be uploaded to Roboflow Deploy (i.e., run inference on custom model on a persistant, auto-scaling, cloud API).

In [ ]:
PROJECT_VERSION.deploy(model_type = "yolov8", model_path = TRAIN_DIR)

## Cleanup
Garbage collection and cleanup once done.

In [1]:
from torch.cuda import empty_cache as empty_cuda_cache
from torch.mps import empty_cache as empty_mps_cache
from gc import collect

def reset_model(model, has_model: bool = True) -> None:
    if is_cuda_available(): empty_cuda_cache()
    elif is_mps_available(): empty_mps_cache()
    
    if has_model: del model

    collect()
    globals().clear()

%reset -f